In [1]:
# for importing data
import os
import qiime2
import numpy as np
import pandas as pd
from skbio import DistanceMatrix, OrdinationResults
from scripts.helper import temporal_plot
pd.options.mode.chained_assignment = None  # default='warn'

# models
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from inspect import signature
from sklearn import preprocessing
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, f1_score, recall_score, precision_score, roc_auc_score
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, ShuffleSplit

# plotting
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')
%matplotlib inline

In [2]:
# all the loadings
loadings_all = {}

# get pcoa metric types
pcoa_name = {'Bray-Curtis':'../../data/DIABIMMUNE-Qiita-11884/q2-analysis/core-metrics-results/bray_curtis_pcoa_results.qza',
             'Jaccard':'../../data/DIABIMMUNE-Qiita-11884/q2-analysis/core-metrics-results/jaccard_pcoa_results.qza',
             'UniFrac':'../../data/DIABIMMUNE-Qiita-11884/q2-analysis/core-metrics-results/unweighted_unifrac_pcoa_results.qza',
             'W-UniFrac':'../../data/DIABIMMUNE-Qiita-11884/q2-analysis/core-metrics-results/weighted_unifrac_pcoa_results.qza',
             'Aitchison':'../../data/DIABIMMUNE-Qiita-11884/q2-analysis/core-metrics-results/aitchison_pcoa_results.qza'}

# import matadata 
mf = pd.read_csv('../../data/DIABIMMUNE-Qiita-11884/metadata-matched.tsv',sep='\t',index_col=0)

mf['delivery'] = mf['Birth Mode:']
for k,loc in pcoa_name.items():
    ord_tmp = qiime2.Artifact.load(loc).view(OrdinationResults)
    ord_tmp = ord_tmp.samples.astype(float)
    ord_tmp_merged = pd.concat([ord_tmp,mf], axis=1, sort=False)
    loadings_all[k+' PCoA'] = ord_tmp_merged.groupby('host_subject_id').aggregate({**{'delivery':'first'},
                                                                                   **{k:'sum' 
                                                                                      for k in ord_tmp.columns}})

# import matadata 
mode_map = {k:v for k,v in zip(mf.host_subject_id,mf.delivery)}
ctf_sub = qiime2.Artifact.load('../../data/DIABIMMUNE-Qiita-11884/q2-analysis/ctf-results/state_subject_ordination.qza')
ctf_sub = ctf_sub.view(qiime2.Metadata).to_dataframe()
ctf_sub = ctf_sub.rename({'PC1':0,'PC2':1,'PC3':2}, axis=1)
ctf_sub['delivery'] = [mode_map[ind] for ind in ctf_sub.subject_id]
ctf_sub = ctf_sub.groupby('subject_id').aggregate({**{'delivery':'first'},**{k:'sum' for k in range(2)}})
loadings_all['CTF PCoA'] = ctf_sub


/Users/cmartino/miniconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/cmartino/miniconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/cmartino/miniconda3/envs/qiime2-2019.7/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/cmarti

In [5]:

res_PR = {}

#for months in range(1,34):
# Data IO and generation
for method, learndf in loadings_all.items():
    y_real = []
    y_proba = []
    
    X = learndf[list(set(learndf.columns) \
                     - set(['delivery']))].values
    X = StandardScaler().fit_transform(X)
    y = list(learndf['delivery'].values)

    # encode 
    le = preprocessing.LabelEncoder()
    le.fit(y)
    y  = le.transform(y) 
    random_state = np.random.RandomState(42)

    ssplt = StratifiedShuffleSplit(n_splits=100,
                         test_size=0.4,
                         random_state=random_state)
    ssplt.get_n_splits(X, y)
    
    for fold_, (train_index, test_index) in enumerate(ssplt.split(X, y)):
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Create a simple classifier
        classifier = KNeighborsClassifier(n_neighbors=5)
        classifier.fit(X_train, y_train)
        y_score = classifier.predict_proba(X_test)[:,1]
        y_hat = classifier.predict(X_test)

        average_precision = average_precision_score(y_test, y_score)
        precision, recall, _ = precision_recall_curve(y_test, y_score)
        auc_ = roc_auc_score(y_test, y_hat)
        
        y_real.append(y_test)
        y_proba.append(y_score)
        
        res_PR[(method, fold_)] = [average_precision, auc_]

    y_real = np.concatenate(y_real)
    y_proba = np.concatenate(y_proba)
    precision, recall, _ = precision_recall_curve(y_real, y_proba)
    average_precision = average_precision_score(y_real, y_proba)


In [6]:
# NOTE: AUC is funky because of class imblance
PRresdf = pd.DataFrame(res_PR, ['AUPR','AUC']).fillna(0).T.reset_index()
PRresdf.columns = ['Metric', 'Fold', 'AUPR', 'AUC']
PRresdf = pd.DataFrame(res_PR, ['AUPR','AUC']).fillna(0).T.reset_index()
PRresdf.columns = ['Metric', 'Fold', 'AUPR', 'AUC']
PRresdf_mean = PRresdf.groupby('Metric').mean()[['AUPR']]
PRresdf_mean['std'] = PRresdf.groupby('Metric').sem()[['AUPR']]
PRresdf_mean = round(PRresdf_mean,3)
PRresdf_mean = PRresdf_mean.astype(str)
PRresdf_mean['AUPR_mean'] = PRresdf_mean['AUPR'] + ' ± ' + PRresdf_mean['std']
PRresdf_mean[['AUPR_mean']].to_csv('../../results/DIABIMMUNE-AUPR.tsv',sep='\t')
PRresdf_mean


,AUPR,std,AUPR_mean
Metric,,,
Aitchison PCoA,0.882,0.002,0.882 ± 0.002
Bray-Curtis PCoA,0.883,0.002,0.883 ± 0.002
CTF PCoA,0.97,0.004,0.97 ± 0.004
Jaccard PCoA,0.887,0.003,0.887 ± 0.003
UniFrac PCoA,0.87,0.001,0.87 ± 0.001
W-UniFrac PCoA,0.877,0.003,0.877 ± 0.003


In [7]:
from collections import Counter
print(Counter(le.inverse_transform(y_train)))
print(Counter(le.inverse_transform(y_test)))

Counter({'Vaginal': 22, 'Cesarean': 2})
Counter({'Vaginal': 14, 'Cesarean': 2})
